In [5]:
import pandas as pd
import psycopg2

In [19]:
business_raw = pd.read_csv('csv_files/yelp_academic_dataset_business.csv')


In [20]:
business_raw.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,2818 E Camino Acequia Drive,{},1SWheh84yJXfytovILXOAQ,"Golf, Active Life",Phoenix,NaN,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,30 Eglinton Avenue W,"{'GoodForMeal': ""{'dessert': False, 'latenight...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"1210 8th Street SW, Unit 220","{'BusinessParking': ""{'garage': False, 'street...",fcXOEZdXYeZqnQ3lGlOXmg,"Local Services, Professional Services, Compute...",Calgary,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,51.041771,-114.081109,Nucleus Information Service,T2R 1L3,5,2.0,AB
3,"6955 N Durango Dr, Ste 1116","{'BusinessParking': ""{'garage': False, 'street...",f2ZWZPENViL92BrFsIgR6w,"Beauty & Spas, Hair Salons",Las Vegas,"{'Monday': '8:30-20:0', 'Tuesday': '8:30-20:0'...",1,36.287312,-115.289540,Amazing Cuts,89149,116,4.5,NV
4,4 E University Dr,{},6KgGE8B1RsR7jc9R5nuH0Q,"American (Traditional), Restaurants",Tempe,NaN,0,33.422192,-111.939615,Ruby Tuesday,85281,9,2.5,AZ


In [15]:
con = psycopg2.connect(database="template1", user="Pialex99", password="", host="127.0.0.1", port="5432")
print("Connection worked")

Connection worked


In [18]:
con.close()

In [16]:
cur = con.cursor()

In [17]:
cur.execute("select * from state")
cur.fetchall()

[(1, 'AZ')]

In [ ]:
cur.execute("insert into state (id, name) values (1, 'AZ')")
con.commit()